***
# neuroCraft Project
***

***
## Sentence Simplification using a Sentence-to-Sentence (seq2seq) Architecture: BERT-to-GPT2 
***
<br>
- Sentence simplification using Hugging Face transformers.
<br>
<br>
- Encoder-Decoder-Model with a combination of BertConfig and GPT2Config as encoder and decoder.

***

# Imports

In [142]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

# Imports
import pandas as pd
import numpy as np

import pickle
import click
import os
import time


import torch
import torch.nn as nn

from torch.utils.data import DataLoader, TensorDataset, Dataset
from collections import Counter

import tqdm
import logging
import gc
import shutil

from transformers import BertTokenizer, GPT2Tokenizer
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from transformers import EncoderDecoderModel, BertConfig, EncoderDecoderConfig, GPT2Tokenizer, BertModel, GPT2Model


***

# Data Loaders

In [143]:
# line reader
def ReadInFile (filename):
    '''
    Utility function designed to read lines from a file and return them as a list of strings.

    - with open(filename) as f:
    Opens the file specified by the filename parameter using a with statement, which ensures that the file is properly closed after its suite finishes, even if an exception occurs.

    - lines = f.readlines():
    Reads all lines from the file f and stores them in the lines variable as a list of strings. Each line from the file becomes an element in the list.

    - lines = [x.strip() for x in lines]:
    Strips any leading or trailing whitespaces (like newline characters) from each line in the lines list using a list comprehension.
    This step ensures that the lines are clean and don't contain any extra spaces.

    - return lines:
    Returns the list of cleaned lines from the file.
    '''
    with open(filename) as f:
        lines = f.readlines()
        lines = [x.strip() for x in lines]
    return lines


# file opener
def open_file(file_path, ref=False):
    '''
    Versatile file opener that can read text files and binary pickle files in Python.

    - reading Text Files:
    If the ref parameter is False or not specified, it assumes the file is a text file.
    It opens the file specified by file_path in read mode ('r') using utf-8 encoding to handle Unicode characters.

    - Loading Pickle Files:
    If ref is set to True, it assumes the file is a binary pickle file (ref_data = pickle.load(open(file_path, 'rb'))) and directly returns the loaded data.

    - Processing Text Files:
    If the file is a text file, it reads lines from the file (sents = f.readlines()) and strips any leading or trailing whitespaces (including newline characters) from each line.
    Each cleaned line is appended to the data list.

    After processing all lines, it returns the list of cleaned lines (data).
    '''
    data = []
    if ref:
        ref_data = pickle.load(open(file_path, 'rb'))
        return ref_data
    else:
        with open(file_path, 'r', encoding="utf8") as f:
            sents = f.readlines()
            for s in sents:
                data.append(s.strip())
        return data

# data loader
def load_dataset(src_path, tgt_path=None, ref_path=None, ref=False):
    '''
    Function serves as a data loader and orchestrator for multiple files for NLP tasks.

    - Loading Source Data:
    Loads the source data from the file specified by src_path using the open_file function without any specific reference handling.

    - Loading Target Data (Optional):
    If a tgt_path is provided, it loads target data using the open_file function and assigns it to the tgt variable.
    This can be used when dealing with tasks like translation and simplification (here) where there are source and target language sentences.

    - Loading Reference Data (Optional):
    If a ref_path is provided, it loads reference data using the open_file function, potentially handling it as a pickle file based on the ref parameter.
    This ref parameter determines whether to load the reference data as a pickle file or text file.

    - Returning Loaded Data:
    returns three variables: src (source data), tgt (target data, if provided), and ref (reference data, if provided)
    '''
    src = open_file(src_path)
    tgt = None
    ref = None
    if tgt_path is not None:
        tgt = open_file(tgt_path)
    if ref_path is not None:
        ref = open_file(ref_path, ref)
    return src, tgt, ref


***

# Data

In [144]:
# CURRENT DATA

"""
1. Training:

Wikilarg dataset comprising of parallel corpus of normal sentences and simple sentences is used to train the model.
The original dataset consists of around 167k English sentence pairs from the Wikipedia articles.
The dataset comprises of mapping of one-to-many, one-to-one and many-to-one sentence pairs.
But the dataset was not suitable for the training without preprocessing.
Upon tokenizing the sentences, sentences having token length of more than 80 were removed keeping the maximum token length of sentences to 80.
The resulting training dataset became 138k from 167k.

2. Validation:

WikiSmall dataset ...

3. Testing:

For the evaluation and testing purpose, TurkCorpus is used.
The dataset consists of 2k manually prepared sentence pairs with 8 reference sentences and 300 sentences for testing purpose which also has 8 reference sentences.
"""
print('')


In [145]:
# keep class for data handling for now

class WikiDataset(Dataset):
    def __init__(self, src_path, tgt_path=None, ref_path=None, ref=False):
        self.src = self.open_file(src_path)
        self.tgt = None
        self.ref = None
        if tgt_path is not None:
            self.tgt = self.open_file(tgt_path)
        if ref_path is not None:
            self.ref = self.open_file(ref_path, ref)

        self.size = len(self.src)

    def __getitem__(self, index):
        if self.tgt is not None and self.ref is not None:
            return self.src[index], self.tgt[index], self.ref[index]
        elif self.tgt is not None:
            return self.src[index], self.tgt[index], None
        else:
            return self.src[index], None, None

    def __len__(self):
        return self.size

    @staticmethod
    def open_file(file_path, ref=False):
        data = []
        if ref:
            ref_data = pickle.load(open(file_path, 'rb'))
            return ref_data

        else:
            with open(file_path, 'r', encoding="utf8") as f:
                sents = f.readlines()
                for s in sents:
                    data.append(s.strip())
            return data


## Training data

In [146]:
# training data
src_train_file = '../dataset/src_train.txt'
tgt_train_file = '../dataset/tgt_train.txt'

# original train data and target train data (no ref data)
src_train, tgt_train, ref_train = load_dataset(src_train_file, tgt_train_file, ref=False)

print('training data (Wikilarg)')
print(pd.DataFrame(src_train).shape)
print(pd.DataFrame(tgt_train).shape)

sample_index = 1000
src_sample = src_train[sample_index]
tgt_sample = tgt_train[sample_index]

print("Source:", src_sample)
print("Target:", tgt_sample)


training data (Wikilarg)
(138413, 1)
(138413, 1)
Source: Quincy in 1767 was the `` north precinct '' of Braintree , Massachusetts .
Target: He was born in Braintree , Massachusetts , in 1767 .


## Validation data

In [147]:
# validation data
src_valid_file = '../dataset/src_valid.txt'
tgt_valid_file = '../dataset/tgt_valid.txt'

# original train data and target train data (no ref data)
src_valid, tgt_valid, ref_valid = load_dataset(src_valid_file, tgt_valid_file, ref=False)

print(' ')
print('validation data (WikiSmall)')
print(pd.DataFrame(src_valid).shape)
print(pd.DataFrame(tgt_valid).shape)

sample_index = 99
src_sample = src_valid[sample_index]
tgt_sample = tgt_valid[sample_index]

print("Source:", src_sample)
print("Target:", tgt_sample)


 
validation data (WikiSmall)
(2000, 1)
(2000, 1)
Source: Since 1980 the senior pastor has been John Piper .
Target: Since 1980 the main pastor has been John Piper .


## Test data

In [148]:
# testing data
src_test_file = '../dataset/src_test.txt'
tgt_test_file = '../dataset/tgt_test.txt'

# original train data and target train data (no ref data)
src_test, tgt_test, ref_test = load_dataset(src_test_file, tgt_test_file, ref=False)

print(' ')
print('test data (Amazon Mechanical Turk workers)')
print(pd.DataFrame(src_test).shape)
print(pd.DataFrame(tgt_test).shape)

sample_index = 10
src_sample = src_test[sample_index]
tgt_sample = tgt_test[sample_index]

print("Source:", src_sample)
print("Target:", tgt_sample)


 
test data (Amazon Mechanical Turk workers)
(359, 1)
(359, 1)
Source: Alessandro ( " Sandro " ) Mazzola ( born 8 November 1942 ) is an Italian former football player .
Target: Alessandro Mazzola is an Italian former football player.


***

# Embeddings

In [149]:
# Set up tokenizers
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_tokenizer.pad_token = gpt2_tokenizer.unk_token


In [150]:
# function for encoding the text
def encode_batch(batch, max_len=80):
    '''
    Inputs
    - batch:
        This function expects a list containing two elements:
        - The first element is a string representing the source text.
        - The second element is a string representing the target text.

    Tokenization
    - src_tokens:
    The source text (first element of batch) is tokenized using the BERT tokenizer (bert_tokenizer).
    It is processed to generate tokens (input_ids) and an attention mask for the source text.
    BERT tokenization includes adding special tokens, padding to a maximum length, and truncating if needed.

    - tgt_tokens:
    The target text (second element of batch) is tokenized using the GPT-2 tokenizer (gpt2_tokenizer).
    Similar to the source, tokens (input_ids) and an attention mask for the target text are generated.

    Creating Labels
    - labels: This is created from the tgt_tokens.input_ids tensor.
    It's used for calculating loss during training.
    The tgt_tokens.attention_mask is used to identify where the padding is in the target tokens and sets those positions in the labels tensor to -100.

    Output
    The function returns five values:
    - src_tokens.input_ids:
    Tensor containing tokenized representation of the source text.
    - src_tokens.attention_mask:
    Tensor containing attention mask for the source text.
    - tgt_tokens.input_ids:
    Tensor containing tokenized representation of the target text.
    - tgt_tokens.attention_mask:
    Tensor containing attention mask for the target text.

    Labels:
    Tensor containing the labels for the target text, modified with -100 in places corresponding to padding.
    '''
    src_tokens = bert_tokenizer(batch[0], max_length=max_len, add_special_tokens=True,
                                return_token_type_ids=False, padding="max_length", truncation=True,
                                return_attention_mask=True, return_tensors="pt")

    tgt_tokens = gpt2_tokenizer(batch[1], max_length=max_len, add_special_tokens=True,
                                return_token_type_ids=False, padding="max_length", truncation=True,
                                return_attention_mask=True, return_tensors="pt")

    labels = tgt_tokens.input_ids.clone()
    labels[tgt_tokens.attention_mask == 0] = -100

    return src_tokens.input_ids, src_tokens.attention_mask, tgt_tokens.input_ids, tgt_tokens.attention_mask, labels

# function for decoding the tokenized sentences to human readable text
def decode_sent_tokens(data):
    '''
    Takes a list of tokenized sentences (data) and decodes them back into human-readable sentences.

    Inputs:
    - data:
    A list containing tokenized sentences.

    Functionality:
    It iterates through each tokenized sentence in the data list.
    For each tokenized sentence, it uses the GPT-2 tokenizer (gpt2_tokenizer) to decode the tokens into a human-readable sentence (s),
    skipping special tokens and cleaning up tokenization spaces.
    The decoded sentences (s) are added to a list (sents_list).

    Output:
    The function returns a list (sents_list) containing the decoded sentences
    '''
    sents_list = []
    for sent in data:
        s = gpt2_tokenizer.decode(sent, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        sents_list.append(s)

    return sents_list

# function to tokenize input sentences for downstream processing
def get_sent_tokens(sents):
    '''
    Tokenizes input sentences and prepares them for downstream processing, e.g., for model input or further manipulation.

    Inputs:
    - sents:
    A list or string containing sentences.

    Functionality:
    Tokenizes the input sentences using the GPT-2 tokenizer (gpt2_tokenizer) with specific settings:
    - Adding special tokens.
    - Not returning token type IDs.
    - Truncating sequences if needed.
    - Padding to the longest sequence in the batch.
    - Not returning attention masks (only input tensors are returned).

    Output:
    returns a list (ref) containing lists of tokenized sentences, where each inner list represents the tokenized form of a sentence from the input.
    '''
    ref = []
    tokens = gpt2_tokenizer(sents, add_special_tokens=True,
                            return_token_type_ids=False, truncation=True, padding="longest",
                            return_attention_mask=False, return_tensors="pt")

    for tok in tokens.input_ids.tolist():
        ref.append([tok])

    return ref


***

# Evaluation Metrics

## SARI (System-level Automatic Readability Index) Score

In [151]:
# implementing function for calculating sari -gram F1 scores (SS task)

def SARIngram(sgrams, cgrams, rgramslist, numref):
    '''
    Function calculating the SARI (System-level Automatic Readability Index) score for a given set of n-grams.
    The SARI score assesses the quality of simplified sentences compared to both the original and reference sentences.

    Counting n-grams:
    The function takes as input three types of n-grams: sgrams (simplified sentence n-grams), cgrams (candidate n-grams), and rgramslist (a list of reference sentence n-grams).
    It creates counters to count occurrences of each n-gram type in these inputs.

    Handling Repetitions:
    It adjusts the counts for sgrams and cgrams by multiplying them by numref, which represents the number of reference sentences for comparison.

    Calculating Scores for Three Aspects (Keep, Deletion, Addition):
    - Keep: Compares n-grams present in both the simplified and candidate sentences against those in the reference sentences.
    - Deletion: Looks at n-grams in the simplified sentences that are absent in the candidate sentences and compares them against reference n-grams.
    - Addition: Considers n-grams in the candidate sentences not present in the simplified sentences and compares them against reference n-grams.

    Precision and Recall Calculations:
    For each aspect (Keep, Deletion, Addition), it calculates precision and recall scores based on the counts of n-grams.

    SARI Score Calculation:
    It computes the final SARI score using the precision and recall scores for Keep, Deletion, and Addition aspects.

    Return:
    Function returns a tuple containing the SARI score, precision score for deletions, and the score for additions.
    '''
    rgramsall = [rgram for rgrams in rgramslist for rgram in rgrams]
    rgramcounter = Counter(rgramsall)

    sgramcounter = Counter(sgrams)
    sgramcounter_rep = Counter()
    for sgram, scount in sgramcounter.items():
        sgramcounter_rep[sgram] = scount * numref

    cgramcounter = Counter(cgrams)
    cgramcounter_rep = Counter()
    for cgram, ccount in cgramcounter.items():
        cgramcounter_rep[cgram] = ccount * numref


    # KEEP
    keepgramcounter_rep = sgramcounter_rep & cgramcounter_rep
    keepgramcountergood_rep = keepgramcounter_rep & rgramcounter
    keepgramcounterall_rep = sgramcounter_rep & rgramcounter

    keeptmpscore1 = 0
    keeptmpscore2 = 0
    for keepgram in keepgramcountergood_rep:
        keeptmpscore1 += keepgramcountergood_rep[keepgram] / keepgramcounter_rep[keepgram]
        keeptmpscore2 += keepgramcountergood_rep[keepgram] / keepgramcounterall_rep[keepgram]
        #print "KEEP", keepgram, keepscore, cgramcounter[keepgram], sgramcounter[keepgram], rgramcounter[keepgram]
    keepscore_precision = 0
    if len(keepgramcounter_rep) > 0:
    	keepscore_precision = keeptmpscore1 / len(keepgramcounter_rep)
    keepscore_recall = 0
    if len(keepgramcounterall_rep) > 0:
    	keepscore_recall = keeptmpscore2 / len(keepgramcounterall_rep)
    keepscore = 0
    if keepscore_precision > 0 or keepscore_recall > 0:
        keepscore = 2 * keepscore_precision * keepscore_recall / (keepscore_precision + keepscore_recall)


    # DELETION
    delgramcounter_rep = sgramcounter_rep - cgramcounter_rep
    delgramcountergood_rep = delgramcounter_rep - rgramcounter
    delgramcounterall_rep = sgramcounter_rep - rgramcounter
    deltmpscore1 = 0
    deltmpscore2 = 0
    for delgram in delgramcountergood_rep:
        deltmpscore1 += delgramcountergood_rep[delgram] / delgramcounter_rep[delgram]
        deltmpscore2 += delgramcountergood_rep[delgram] / delgramcounterall_rep[delgram]
    delscore_precision = 0
    if len(delgramcounter_rep) > 0:
    	delscore_precision = deltmpscore1 / len(delgramcounter_rep)
    delscore_recall = 0
    if len(delgramcounterall_rep) > 0:
    	delscore_recall = deltmpscore1 / len(delgramcounterall_rep)
    delscore = 0
    if delscore_precision > 0 or delscore_recall > 0:
        delscore = 2 * delscore_precision * delscore_recall / (delscore_precision + delscore_recall)


    # ADDITION
    addgramcounter = set(cgramcounter) - set(sgramcounter)
    addgramcountergood = set(addgramcounter) & set(rgramcounter)
    addgramcounterall = set(rgramcounter) - set(sgramcounter)

    addtmpscore = 0
    for addgram in addgramcountergood:
        addtmpscore += 1

    addscore_precision = 0
    addscore_recall = 0
    if len(addgramcounter) > 0:
    	addscore_precision = addtmpscore / len(addgramcounter)
    if len(addgramcounterall) > 0:
    	addscore_recall = addtmpscore / len(addgramcounterall)
    addscore = 0
    if addscore_precision > 0 or addscore_recall > 0:
        addscore = 2 * addscore_precision * addscore_recall / (addscore_precision + addscore_recall)

    return (keepscore, delscore_precision, addscore)



def SARIsent(ssent, csent, rsents):
    '''
    This function computes the System-level Automatic Readability Index (SARI) score for a set of sentences.

    Tokenizing Sentences:

    Takes in three inputs
    - ssent (simplified sentence)
    - csent (candidate sentence)
    - and rsents (a list of reference sentences)
    The function begins by splitting these sentences into unigrams, bigrams, trigrams, and quadgrams
    (s1grams, s2grams, s3grams, s4grams, c1grams, c2grams, c3grams, c4grams, r1gramslist, r2gramslist, r3gramslist, r4gramslist)
    using spaces as delimiters.

    Creating n-grams from Reference Sentences:

    Creates n-grams
    (unigrams, bigrams, trigrams, and quadgrams)
    from the reference sentences (rsents) and stores them in separate lists.

    Calculating Scores for Different n-gram Levels:

    For each level of n-grams (from unigrams to quadgrams) in both the simplified and candidate sentences,
    it calls the SARIngram function to compute scores for keep, delete, and add operations for each level of n-grams.

    Aggregating Scores:
    It averages the scores for keep, delete, and add operations across different n-gram levels to get average keep, delete, and add scores.

    Computing the Final SARI Score:
    Computes the final SARI score by averaging the average keep, delete, and add scores, and then returns the final score.
    '''
    numref = len(rsents)

    s1grams = ssent.lower().split(" ")
    c1grams = csent.lower().split(" ")
    s2grams = []
    c2grams = []
    s3grams = []
    c3grams = []
    s4grams = []
    c4grams = []

    r1gramslist = []
    r2gramslist = []
    r3gramslist = []
    r4gramslist = []
    for rsent in rsents:
        r1grams = rsent.lower().split(" ")
        r2grams = []
        r3grams = []
        r4grams = []
        r1gramslist.append(r1grams)
        for i in range(0, len(r1grams)-1) :
            if i < len(r1grams) - 1:
                r2gram = r1grams[i] + " " + r1grams[i+1]
                r2grams.append(r2gram)
            if i < len(r1grams)-2:
                r3gram = r1grams[i] + " " + r1grams[i+1] + " " + r1grams[i+2]
                r3grams.append(r3gram)
            if i < len(r1grams)-3:
                r4gram = r1grams[i] + " " + r1grams[i+1] + " " + r1grams[i+2] + " " + r1grams[i+3]
                r4grams.append(r4gram)
        r2gramslist.append(r2grams)
        r3gramslist.append(r3grams)
        r4gramslist.append(r4grams)

    for i in range(0, len(s1grams)-1) :
        if i < len(s1grams) - 1:
            s2gram = s1grams[i] + " " + s1grams[i+1]
            s2grams.append(s2gram)
        if i < len(s1grams)-2:
            s3gram = s1grams[i] + " " + s1grams[i+1] + " " + s1grams[i+2]
            s3grams.append(s3gram)
        if i < len(s1grams)-3:
            s4gram = s1grams[i] + " " + s1grams[i+1] + " " + s1grams[i+2] + " " + s1grams[i+3]
            s4grams.append(s4gram)

    for i in range(0, len(c1grams)-1) :
        if i < len(c1grams) - 1:
            c2gram = c1grams[i] + " " + c1grams[i+1]
            c2grams.append(c2gram)
        if i < len(c1grams)-2:
            c3gram = c1grams[i] + " " + c1grams[i+1] + " " + c1grams[i+2]
            c3grams.append(c3gram)
        if i < len(c1grams)-3:
            c4gram = c1grams[i] + " " + c1grams[i+1] + " " + c1grams[i+2] + " " + c1grams[i+3]
            c4grams.append(c4gram)

    (keep1score, del1score, add1score) = SARIngram(s1grams, c1grams, r1gramslist, numref)
    (keep2score, del2score, add2score) = SARIngram(s2grams, c2grams, r2gramslist, numref)
    (keep3score, del3score, add3score) = SARIngram(s3grams, c3grams, r3gramslist, numref)
    (keep4score, del4score, add4score) = SARIngram(s4grams, c4grams, r4gramslist, numref)
    avgkeepscore = sum([keep1score,keep2score,keep3score,keep4score])/4
    avgdelscore = sum([del1score,del2score,del3score,del4score])/4
    avgaddscore = sum([add1score,add2score,add3score,add4score])/4
    finalscore = (avgkeepscore + avgdelscore + avgaddscore ) / 3

    return finalscore


In [152]:
# final computation function for sentence level SARI
def compute_sari(norm, pred_tensor, ref):
    '''
    computes the Sentence-level SARI score for a set of predicted sentences compared to their corresponding reference sentences.

    Decoding Predicted Sentences:
    decodes the predicted sentences from the pred_tensor using the tokenizer.
    This step converts tokenized or numerical representations back into human-readable sentences.

    SARI Score Calculation:
    The function initializes a score variable to accumulate the SARI scores for individual sentences.
    It iterates over each step (index) and corresponding items in the ref list.

    For each step:
    Computes the SARI score using sari.SARIsent(norm[step], pred[step], item).
    This compares the normalized (norm[step]) predicted sentence with the reference sentence (item) and computes the SARI score.
    Adds up the individual SARI scores to the score variable.

    Normalization and Return:
    The final computed score is divided by the TRAIN_BATCH_SIZE to normalize the score.

    Returns the normalized SARI score, which is the cumulative SARI score of the predicted sentences against their reference sentences, divided by the batch size.
    '''
    pred = decode_sent_tokens(pred_tensor)
    score = 0
    for step, item in enumerate(ref):
        score += SARIsent(norm[step], pred[step], item)
    return score/TRAIN_BATCH_SIZE


## BLEU Score

In [153]:
# computation function
def compute_bleu_score(logits, labels):
    '''
    This function takes predicted sequences (logits) and their corresponding reference sequences (labels),
    tokenizes the reference sentences, and computes a weighted BLEU score using NLTK's corpus_bleu function.
    The BLEU score is a metric commonly used to evaluate the quality of machine-translated text against reference translations,
    considering n-gram precision with specified weights.

    Tokenization of Reference Sentences:
    It retrieves the reference sentences by tokenizing the labels input.
    This step involves converting human-readable sentences into tokens suitable for BLEU score computation.

    Weighted BLEU Score Calculation:
    weights are assigned to n-gram precision values (e.g., 1-gram precision and 2-gram precision).

    BLEU Score Calculation:
    computes the BLEU score using the corpus_bleu function from NLTK's nltk.translate.bleu_score module.
    The refs variable contains the reference sentences (tokenized), and logits.tolist() contain the predicted sequences, converted to a list format.
    The smoothing_function parameter applies a smoothing method to handle cases where certain n-grams are missing in the predicted sequences.
    The computed BLEU score based on these inputs is stored in the score variable.

    Returns the computed BLEU score as the output of the function.
    '''
    refs = get_sent_tokens(labels)
    weights = (1.0/2.0, 1.0/2.0, )
    score = corpus_bleu(refs, logits.tolist(), smoothing_function=SmoothingFunction(epsilon=1e-10).method1, weights=weights)
    return score


***

# Training Evaluation

In [154]:
# Evaluation after training (unseen test data)
def evaluate(data_loader, e_loss):
    '''
    Performs evaluation on the trained model using the provided data loader (data_loader).
    Assesses the model's performance in terms of loss, BLEU score, and SARI score on the test dataset.

    Setting Model Evaluation Mode
    - was_training = model.training:
    Stores the model's current training state.

    - model.eval():
    Puts the model in evaluation mode, which disables dropout and batch normalization layers as they behave differently during training and evaluation.

    - initialization:
    Initializes variables for evaluation metrics (eval_loss, bleu_score, sari_score) to keep track of loss, BLEU score, and SARI score.

    LogSoftmax and No Gradient Calculation
    - softmax = nn.LogSoftmax(dim=-1): Initializes a LogSoftmax layer for computing probabilities along the last dimension (-1) of the tensor.
    - with torch.no_grad():: Temporarily disables gradient calculation for efficiency during evaluation.

    Iterating Over Data Loader Batches:
    Loops over batches in the data_loader.
    Encodes the batch data using the tokenizer.
    Passes the encoded tensors to the model to get predictions (logits) and calculate the loss.
    Computes the BLEU score and SARI score using the compute_bleu_score and compute_sari functions,
    comparing the model's predicted outputs with the ground truth labels.
    Updates the evaluation metrics (eval_loss, bleu_score, sari_score) by averaging across batches.

    Restoring Model State and Return:
    Restores the model to its original training state (model.train()) if it was in training mode before evaluation.

    Returns the evaluated loss (eval_loss), BLEU score (bleu_score), and SARI score (sari_score) as a tuple.
    '''
    was_training = model.training
    model.eval()
    eval_loss = e_loss
    bleu_score = 0
    sari_score = 0
    softmax = nn.LogSoftmax(dim = -1)

    with torch.no_grad():
        for step, batch in enumerate(data_loader):
            src_tensors, src_attn_tensors, tgt_tensors, tgt_attn_tensors, labels = encode_batch(batch)
            loss, logits = model(input_ids = src_tensors.to(device),
                            decoder_input_ids = tgt_tensors.to(device),
                            attention_mask = src_attn_tensors.to(device),
                            decoder_attention_mask = tgt_attn_tensors.to(device),
                            labels = labels.to(device))[:2]
            outputs = softmax(logits)
            score = compute_bleu_score(torch.argmax(outputs, dim=-1), batch[1])
            s_score = compute_sari(batch[0], torch.argmax(outputs, dim=-1), batch[2])
            if step == 0:
                eval_loss = loss.item()
                bleu_score = score
                sari_score = s_score
            else:
                eval_loss = (1/2.0)*(eval_loss + loss.item())
                bleu_score = (1/2.0)* (bleu_score+score)
                sari_score = (1/2.0)* (sari_score+s_score)

    if was_training:
        model.train()

    return eval_loss, bleu_score, sari_score


***

## Model Checkpoints

In [155]:
# Load existing Checkpoints
def load_checkpt(checkpt_path, optimizer=None):
    '''
    Loading a checkpoint file containing model weights, optimizer state, evaluation loss, and epoch information. Let's break down its functionality:

    Loading Checkpoint
    - checkpoint = torch.load(checkpt_path):
    Loads the checkpoint file from the specified checkpt_path using PyTorch's torch.load() function.

    Loading Model and Optimizer States
    Depending on the device being used ("cpu" or GPU), it loads the model's state and optimizer's state from the checkpoint file.
    If the device is "cpu," it maps both the model and optimizer states to the CPU using map_location=torch.device("cpu").
    If the device is a GPU, it loads the model and optimizer states directly without any mapping.

    Retrieving Evaluation Loss and Epoch:
    Extracts the evaluation loss (eval_loss) and epoch information (epoch) from the loaded checkpoint.

    Returns the loaded optimizer (if provided and updated), evaluation loss, and epoch.
    '''
    checkpoint = torch.load(checkpt_path)
    if device == "cpu":
        model.load_state_dict(checkpoint["model_state_dict"], map_location=torch.device("cpu"))
        if optimizer is not None:
            optimizer.load_state_dict(checkpoint["optimizer_state_dict"], map_location=torch.device("cpu"))
    else:
        model.load_state_dict(checkpoint["model_state_dict"])
        if optimizer is not None:
            optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

    eval_loss = checkpoint["eval_loss"]
    epoch = checkpoint["epoch"]

    return optimizer, eval_loss, epoch


In [156]:
# Save Training Checkpoint
def save_model_checkpt(state, is_best, check_pt_path, best_model_path):
    '''
    Saves model checkpoints at regular intervals during training and, if specified,
    also keeps a copy of the best-performing model in a separate file.
    This practice allows for the restoration of model states,
    retraining from specific points,
    or retrieving the best model for deployment or further evaluation.
    '''
    f_path = check_pt_path
    torch.save(state, f_path)

    if is_best:
        best_fpath = best_model_path
        shutil.copyfile(f_path, best_fpath)


In [157]:
def evaluate_model(model, data_loader, criterion, target_tokenizer, ref_sentences):
    model.eval()
    total_loss = 0.0
    total_batches = 0
    references = []
    hypotheses = []

    with torch.no_grad():
        for batch in data_loader:
            batch_source = batch[0]
            batch_target = batch[1]

            outputs = model(input_ids=batch_source, decoder_input_ids=batch_target)

            logits_flat = outputs.logits.view(-1, outputs.logits.size(-1))
            target_flat = batch_target.view(-1)

            loss = criterion(logits_flat, target_flat)
            total_loss += loss.item()
            total_batches += 1

            predicted_ids = outputs.logits.argmax(-1)
            predicted_sentences = [target_tokenizer.decode(ids, skip_special_tokens=True) for ids in predicted_ids]
            target_sentences = [target_tokenizer.decode(ids, skip_special_tokens=True) for ids in batch_target]

            references.extend([sent.split() for sent in target_sentences])
            hypotheses.extend([sent.split() for sent in predicted_sentences])

    avg_loss = total_loss / total_batches

    # Calculate BLEU score using your function
    #bleu_score = compute_bleu_score(torch.argmax(outputs, dim=-1), batch[1])
    logits = outputs.logits  # Extract the logits from Seq2SeqLMOutput
    predicted_ids = torch.argmax(logits, dim=-1)  # Get the predicted IDs
    bleu_score = compute_bleu_score(predicted_ids, batch[1])  # Compute BLEU score

    # Calculate SARI score
    sari_scores = []
    for idx, gen_sent in enumerate(hypotheses):
        sari_score = compute_sari(ref_sentences[idx], gen_sent)
        sari_scores.append(sari_score)

    avg_sari_score = sum(sari_scores) / len(sari_scores)

    return avg_loss, bleu_score, avg_sari_score


***

# Prepare for Training

## Process validation data

In [ ]:
# prepare training dataset


In [158]:
# prepare validation dataset

# function to load evaluation data
def load_eval_data(src_path, tgt_path):
    src_data = open_file(src_path)
    tgt_data = open_file(tgt_path)
    return src_data, tgt_data

# evaluation data paths
src_eval_file = '../dataset/src_valid.txt'
tgt_eval_file = '../dataset/tgt_valid.txt'

# load evaluation data
src_eval, tgt_eval = load_eval_data(src_eval_file, tgt_eval_file)

# prepare evaluation data tensors using encode_batch function
source_eval_data_encoded = encode_batch((src_eval))
target_eval_data_encoded = encode_batch((tgt_eval))

# create evaluation data loader
eval_dataset = TensorDataset(*source_eval_data_encoded, *target_eval_data_encoded)
eval_data_loader = DataLoader(eval_dataset, batch_size=3, shuffle=False)

# ref sentences
# load data from the pickle file
with open('../dataset/ref_valid.pkl', 'rb') as file:
    ref_sentences = pickle.load(file)


***

# Model Training

In [159]:
# WORK IN PROGRESS

'''
To be implemented:
- Logging
- Save trained Model
- Evaluation Metric (SARI)
- Prediction
'''

print('')


In [160]:
 # function to invoke the training
def train_model(start_epoch, eval_loss, loaders, optimizer, check_pt_path, best_model_path):
    best_eval_loss = eval_loss
    print("Model training started...")
    for epoch in range(start_epoch, N_EPOCHS):
        print(f"Epoch {epoch} running...")
        epoch_start_time = time.time()
        epoch_train_loss = 0
        epoch_eval_loss = 0
        model.train()
        for step, batch in enumerate(loaders[0]):
            src_tensors, src_attn_tensors, tgt_tensors, tgt_attn_tensors, labels = encode_batch(batch)
            optimizer.zero_grad()
            model.zero_grad()
            loss = model(input_ids = src_tensors.to(device),
                            decoder_input_ids = tgt_tensors.to(device),
                            attention_mask = src_attn_tensors.to(device),
                            decoder_attention_mask = tgt_attn_tensors.to(device),
                            labels = labels.to(device))[0]
            if step == 0:
                epoch_train_loss = loss.item()
            else:
                epoch_train_loss = (1/2.0)*(epoch_train_loss + loss.item())

            loss.backward()
            optimizer.step()

            if (step+1) % LOG_EVERY == 0:
                print(f'Epoch: {epoch} | iter: {step+1} | avg. train loss: {epoch_train_loss} | time elapsed: {time.time() - epoch_start_time}')
                logging.info(f'Epoch: {epoch} | iter: {step+1} | avg. train loss: {epoch_train_loss} | time elapsed: {time.time() - epoch_start_time}')

        eval_start_time = time.time()
        epoch_eval_loss, bleu_score, sari_score = evaluate(loaders[1], epoch_eval_loss)
        epoch_eval_loss = epoch_eval_loss/TRAIN_BATCH_SIZE
        print(f'Completed Epoch: {epoch} | avg. eval loss: {epoch_eval_loss:.5f} | blue score: {bleu_score} | Sari score: {sari_score} | time elapsed: {time.time() - eval_start_time}')
        logging.info(f'Completed Epoch: {epoch} | avg. eval loss: {epoch_eval_loss:.5f} | blue score: {bleu_score}| Sari score: {sari_score} | time elapsed: {time.time() - eval_start_time}')

        check_pt = {
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'eval_loss': epoch_eval_loss,
            'sari_score': sari_score,
            'bleu_score': bleu_score
        }
        check_pt_time = time.time()
        print("Saving Checkpoint.......")
        if epoch_eval_loss < best_eval_loss:
            print("New best model found")
            logging.info(f"New best model found")
            best_eval_loss = epoch_eval_loss
            save_model_checkpt(check_pt, True, check_pt_path, best_model_path)
        else:
            save_model_checkpt(check_pt, False, check_pt_path, best_model_path)
        print(f"Checkpoint saved successfully with time: {time.time() - check_pt_time}")
        logging.info(f"Checkpoint saved successfully with time: {time.time() - check_pt_time}")

        gc.collect()
        torch.cuda.empty_cache()


In [161]:
# train function
def train(base_path="../",
          src_train="dataset/src_train.txt",
          tgt_train="dataset/tgt_train.txt",
          src_valid="dataset/src_valid.txt",
          tgt_valid="dataset/tgt_valid.txt",
          ref_valid="dataset/ref_valid.pkl",
          best_model="best_model/model.pt",
          checkpoint_path="checkpoint/model_ckpt.pt", seed=123):

    print("Loading datasets...")
    train_dataset = WikiDataset(base_path + src_train, base_path + tgt_train)
    valid_dataset = WikiDataset(base_path + src_valid, base_path + tgt_valid, base_path + ref_valid, ref=True)
    print("Dataset loaded successfully")

    train_dl = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
    valid_dl = DataLoader(valid_dataset, batch_size=TRAIN_BATCH_SIZE, collate_fn=collate_fn, shuffle=True)

    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=3e-5)

    if os.path.exists(base_path + checkpoint_path):
        optimizer, eval_loss, start_epoch = load_checkpt(base_path + checkpoint_path, optimizer)
        print(f"Loading model from checkpoint with start epoch: {start_epoch} and loss: {eval_loss}")
        logging.info(f"Model loaded from saved checkpoint with start epoch: {start_epoch} and loss: {eval_loss}")

    return "set-up done"


In [162]:
################### constants
TRAIN_BATCH_SIZE = 3 # kept as in the source paper
N_EPOCHS = 20 # they trained with 20 epochs
max_token_len = 80 # can be configured as needed (Classification data?)
LOG_EVERY = 10000

################### encode source and target data using encode_batch function
source_train_data_encoded = encode_batch((src_train))
target_train_data_encoded = encode_batch((tgt_train))

################### logging during training
'''
Configures logging settings using the logging.basicConfig method,
to write log messages to a file named "log_file.log" with the INFO level.
format: timestamp, log level, message
'''
logging.basicConfig(filename="log_file.log", level=logging.INFO,
                format="%(asctime)s:%(levelname)s: %(message)s")
CONTEXT_SETTINGS = dict(help_option_names = ['-h', '--help'])

################### collate data batches
def collate_fn(batch):
    '''
    custom collation function used in PyTorch for data preprocessing in DataLoader
    It defines how individual samples within a batch are combined and transformed
    before being fed into the model during training and evaluation.
    '''
    data_list, label_list, ref_list = [], [], []
    for _data, _label, _ref in batch:
        data_list.append(_data)
        label_list.append(_label)
        ref_list.append(_ref)
    return data_list, label_list, ref_list

################### initialize models
'''
Initializes the model as an Encoder-Decoder architecture using the EncoderDecoderModel from Hugging Face's transformers library.
Model combines an encoder (BERT in this case) and a decoder (GPT-2 in this case) for sequence-to-sequence task.
'''
bert_model = BertModel.from_pretrained('bert-base-cased')
gpt2_model = GPT2Model.from_pretrained('gpt2')

################### configuration
max_token_len = 80
start_token_id = bert_tokenizer.cls_token_id
end_token_id = gpt2_tokenizer.eos_token_id

################### initialize the Encoder-Decoder model with cross-attention enabled
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-cased', 'gpt2')

################### update configuration for the model
'''
Updates the configuration for the encoder-decoder model:
- Configure the start and end token IDs for the encoder and decoder
- Set the maximum length for tokenized sequences
- Enable cross-attention in the model (model.config.add_cross_attention = True),
  allowing the decoder to attend to different parts of the encoded input sequence during decoding.
'''
model.config.decoder_start_token_id = start_token_id
model.config.eos_token_id = end_token_id
model.config.max_length = max_token_len
model.config.no_repeat_ngram_size = 3
model.config.add_cross_attention = True

#################### set device
'''
Checks for GPU availability,
selects the appropriate device ("cuda" or "cpu"),
prints the chosen device,
and then moves the model to that device for computation.
TAllows the code to leverage GPU acceleration if a compatible GPU is available,
enhancing computational performance for deep learning tasks.
'''
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} as device")
model.to(device)

################## loss function and optimizer
'''
Loss Function
- nn.CrossEntropyLoss():
Initializes the cross-entropy loss function from the torch.nn module.
Cross-entropy loss is commonly used for multi-class classification tasks, where the model's output represents class probabilities.

Optimizer:
- torch.optim.Adam(model.parameters(), lr=0.001):
Initializes the Adam optimizer from the torch.optim module.
It optimizes the model's parameters (model.parameters()) using the Adam optimization algorithm, a popular variant of stochastic gradient descent (SGD).
'''
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

train(base_path="../",
      src_train="dataset/src_train.txt",
      tgt_train="dataset/tgt_train.txt",
      src_valid="dataset/src_valid.txt",
      tgt_valid="dataset/tgt_valid.txt",
      ref_valid="dataset/ref_valid.pkl",
      best_model="best_model/model.pt",
      checkpoint_path="checkpoint/model_ckpt.pt",
      seed=123)


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.7.crossattention.q_attn.bias', 'h.10.crossattention.c_attn.weight', 'h.7.crossattention.c_proj.weight', 'h.11.ln_cross_attn.bias', 'h.4.crossattention.c_attn.bias', 'h.3.ln_cross_attn.bias', 'h.0.crossattention.c_attn.weight', 'h.9.crossattention.c_attn.weight', 'h.1.crossattention.c_attn.weight', 'h.1.crossattention.q_attn.weight', 'h.0.crossattention.c_attn.bias', 'h.10.crossattention.c_attn.bias', 'h.2.crossattention.c_attn.weight', 'h.8.ln_cross_attn.weight', 'h.0.crossattention.q_attn.bias', 'h.9.ln_cross_attn.weight', 'h.1.crossattention.q_attn.bias', 'h.4.ln_cross_attn.bias', 'h.8.crossattention.c_attn.bias', 'h.9.ln_cross_attn.bias', 'h.0.crossattention.c_proj.weight', 'h.3.crossattention.q_attn.weight', 'h.1.crossattention.c_attn.bias', 'h.7.crossattention.c_attn.weight', 'h.7.crossattention.q_attn.weight', 'h.11.crossattention.q_attn.weight', 'h.6.crossattenti

Using cpu as device
Loading datasets...
Dataset loaded successfully


'set-up done'

In [173]:
train_model(start_epoch=0,
            val_loss=0,
            loaders=load_dataset(src_path="../dataset/src_train.txt",
                                 gt_path="../dataset/tgt_train.txt",
                                 ref=False
                                 ),
            optimizer=optimizer,
            check_pt_path="checkpoint",
            best_model_path="best_model"
            )


Model training started...
Epoch 0 running...


/Users/christine/.pyenv/versions/3.10.6/envs/neuroCraft/lib/python3.10/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:639: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


# Model Testing

In [75]:
# define function for testing the model
def test(base_path="../",
         src_test="dataset/src_test.txt",
         tgt_test="dataset/tgt_test.txt",
         ref_test="dataset/ref_test.pkl",
         best_model="best_model/model.pt"):
    '''
    This function conducts testing of a trained model on a provided test dataset. It follows these steps:

    1. Initialization:
    - Logging and Setup:
    Logs the initiation of the testing process and loads the best model's checkpoint.

    2.*Model Preparation:
    - Model Configuration:
    Sets the model to evaluation mode (`model.eval()`).
    - Dataset Loading:
    Loads the test dataset using paths provided or defaults, creating a `WikiDataset` for testing.

    3. Testing Process:
    - Data Loading:
    Creates a DataLoader for the test dataset with predefined batch size and collation function.
    - Evaluation:
    Evaluates the model on the test dataset, capturing test loss, BLEU score, and SARI score.

    4. Results and Reporting:
    - Metrics Calculation:
    Calculates average evaluation loss, BLEU score, and SARI score, dividing the loss by the batch size.
    - Reporting:
    Prints and logs the calculated metrics and the elapsed time for the testing process.

    5. Completion:
    - inalization:
    Prints "Test Complete!" to indicate the conclusion of the testing phase.
    '''

    print("Testing Model module executing...")
    logging.info("Test module invoked.")

    _, _, _ = load_checkpt(base_path + best_model)
    print("Model loaded.")

    model.eval()
    test_dataset = WikiDataset(base_path + src_test, base_path + tgt_test, base_path + ref_test, ref=True)
    test_dl = DataLoader(test_dataset, batch_size=TRAIN_BATCH_SIZE, collate_fn=collate_fn, shuffle=True)

    test_start_time = time.time()
    test_loss, bleu_score, sari_score = evaluate(test_dl, 0)
    test_loss = test_loss / TRAIN_BATCH_SIZE

    print(f'Avg. eval loss: {test_loss:.5f} | blue score: {bleu_score} | sari score: {sari_score} | time elapsed: {time.time() - test_start_time}')
    logging.info(f'Avg. eval loss: {test_loss:.5f} | blue score: {bleu_score} | sari score: {sari_score} | time elapsed: {time.time() - test_start_time}')

    print("Test Complete!")


# Applying the Model for Simplification of Sentences

In [76]:
def decode(base_path="../",
           src_file="dataset/src_file.txt",
           best_model="best_model/model.pt",
           output="outputs/decoded.txt"):
    '''
     loads a pre-trained model, decodes sentences from a specified source file using the model, and saves the decoded sentences to an output file.
     It uses the tokenizer to encode and decode sentences and conducts these operations sequentially within the function.
    '''
    print("Decoding sentences module executing...")
    logging.info("Decode module invoked.")

    _, _, _ = load_checkpt(base_path + best_model)
    print("Model loaded.")

    model.eval()
    dataset = WikiDataset(base_path + src_file)
    predicted_list = []
    sent_tensors = tokenizer.encode_sent(dataset.src)

    print("Decoding Sentences...")
    for sent in sent_tensors:
        with torch.no_grad():
            predicted = model.generate(sent[0].to(device), attention_mask=sent[1].to(device), decoder_start_token_id=model.config.decoder.decoder_start_token_id)
            predicted_list.append(predicted.squeeze())

    output = tokenizer.decode_sent_tokens(predicted_list)

    with open(base_path + output, "w") as f:
        for sent in output:
            f.write(sent + "\n")

    print("Output file saved successfully.")


# OLD

In [ ]:
# Evaluation of the model during training

# function to evaluate model
def evaluate_model(model, data_loader, criterion, target_tokenizer):
    '''
    Inputs:
    model: The trained seq2seq model to evaluate.
    data_loader: The data loader providing batches of evaluation data. It should contain pairs of source and target sequences.

    criterion: The loss function used for training the model, typically nn.CrossEntropyLoss() or similar.
    target_tokenizer: The tokenizer used for tokenizing the target sequences.

    Functionality:
    Sets the model to evaluation mode (model.eval()).
    Iterates through the evaluation data in batches.
    Passes the source sequences through the model to generate predictions.
    Computes the loss between predicted sequences and actual target sequences.
    Converts model predictions and target sequences from token IDs to text.
    Stores references (actual target sentences) and hypotheses (predicted sentences) to calculate BLEU score.
    Calculates the average loss and BLEU score over the evaluation dataset.

    Outputs:
    avg_loss: Average loss over the evaluation dataset.
    bleu_score: BLEU score indicating the quality of the model's translations (here simplification) compared to the ground truth.
    BLEU (Bilingual Evaluation Understudy) is a metric used to evaluate the quality of machine-translated text against one or more reference translations.
    It's widely used in natural language processing and machine translation tasks, including simplification.
    '''

    model.eval()
    total_loss = 0.0
    total_batches = 0
    references = []
    hypotheses = []

    with torch.no_grad():
        for batch in data_loader:
            batch_source = batch[0]
            batch_target = batch[1]

            outputs = model(input_ids=batch_source, decoder_input_ids=batch_target)

            logits_flat = outputs.logits.view(-1, outputs.logits.size(-1))
            target_flat = batch_target.view(-1)

            loss = criterion(logits_flat, target_flat)
            total_loss += loss.item()
            total_batches += 1

            predicted_ids = outputs.logits.argmax(-1)
            predicted_sentences = [target_tokenizer.decode(ids, skip_special_tokens=True) for ids in predicted_ids]
            target_sentences = [target_tokenizer.decode(ids, skip_special_tokens=True) for ids in batch_target]

            references.extend([sent.split() for sent in target_sentences])
            hypotheses.extend([sent.split() for sent in predicted_sentences])

    avg_loss = total_loss / total_batches

    # Calculate BLEU score with smoothing
    smooth_func = SmoothingFunction().method4
    bleu_score = corpus_bleu(references, hypotheses, smoothing_function=smooth_func)

    return avg_loss, bleu_score


In [68]:
##################################### TRAINING ######################################################

################### constants
TRAIN_BATCH_SIZE = 3 # kept as in the source paper
N_EPOCHS = 20 # they trained with 20 epochs
max_token_len = 80 # can be configured as needed (Classification data?)
LOG_EVERY = 10000

################### encode source and target data using encode_batch function
source_train_data_encoded = encode_batch((src_train))
target_train_data_encoded = encode_batch((tgt_train))

################### logging during training
'''
Configures logging settings using the logging.basicConfig method,
to write log messages to a file named "log_file.log" with the INFO level.
format: timestamp, log level, message
'''
logging.basicConfig(filename="log_file.log", level=logging.INFO,
                format="%(asctime)s:%(levelname)s: %(message)s")
CONTEXT_SETTINGS = dict(help_option_names = ['-h', '--help'])

#################### set device
'''
Checks for GPU availability,
selects the appropriate device ("cuda" or "cpu"),
prints the chosen device,
and then moves the model to that device for computation.
TAllows the code to leverage GPU acceleration if a compatible GPU is available,
enhancing computational performance for deep learning tasks.
'''
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} as device")
model.to(device)

################### collate data batches
def collate_fn(batch):
    '''
    custom collation function used in PyTorch for data preprocessing in DataLoader
    It defines how individual samples within a batch are combined and transformed
    before being fed into the model during training and evaluation.
    '''
    data_list, label_list, ref_list = [], [], []
    for _data, _label, _ref in batch:
        data_list.append(_data)
        label_list.append(_label)
        ref_list.append(_ref)
    return data_list, label_list, ref_list

################### initialize models
'''
Initializes the model as an Encoder-Decoder architecture using the EncoderDecoderModel from Hugging Face's transformers library.
Model combines an encoder (BERT in this case) and a decoder (GPT-2 in this case) for sequence-to-sequence task.
'''
bert_model = BertModel.from_pretrained('bert-base-cased')
gpt2_model = GPT2Model.from_pretrained('gpt2')

################### configuration
max_token_len = 80
start_token_id = bert_tokenizer.cls_token_id
end_token_id = gpt2_tokenizer.eos_token_id

################### initialize the Encoder-Decoder model with cross-attention enabled
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-cased', 'gpt2')

################### update configuration for the model
'''
Updates the configuration for the encoder-decoder model:
- Configure the start and end token IDs for the encoder and decoder
- Set the maximum length for tokenized sequences
- Enable cross-attention in the model (model.config.add_cross_attention = True),
  allowing the decoder to attend to different parts of the encoded input sequence during decoding.
'''
model.config.decoder_start_token_id = start_token_id
model.config.eos_token_id = end_token_id
model.config.max_length = max_token_len
model.config.no_repeat_ngram_size = 3
model.config.add_cross_attention = True

################## loss function and optimizer
'''
Loss Function
- nn.CrossEntropyLoss():
Initializes the cross-entropy loss function from the torch.nn module.
Cross-entropy loss is commonly used for multi-class classification tasks, where the model's output represents class probabilities.

Optimizer:
- torch.optim.Adam(model.parameters(), lr=0.001):
Initializes the Adam optimizer from the torch.optim module.
It optimizes the model's parameters (model.parameters()) using the Adam optimization algorithm, a popular variant of stochastic gradient descent (SGD).
'''
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

################## training loop
for epoch in range(N_EPOCHS):
    running_loss = 0.0
    for i in range(0, len(source_train_data_encoded[0]), TRAIN_BATCH_SIZE):
        batch_source = source_train_data_encoded[0][i:i+TRAIN_BATCH_SIZE]
        batch_target = target_train_data_encoded[0][i:i+TRAIN_BATCH_SIZE]

        optimizer.zero_grad()

        outputs = model(input_ids=batch_source, decoder_input_ids=batch_target)

        logits_flat = outputs.logits.view(-1, outputs.logits.size(-1))
        target_flat = batch_target.view(-1)

        loss = criterion(logits_flat, target_flat)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % LOG_EVERY == 0 and i > 0:
            print(f'Epoch [{epoch + 1}/{N_EPOCHS}], Batch [{i + 1}/{len(source_train_data_encoded[0])}], Loss: {running_loss / LOG_EVERY:.4f}')
            running_loss = 0.0

    # Evaluate
    val_loss, val_bleu = evaluate_model(model, eval_data_loader, criterion, gpt2_tokenizer)
    print(f'Epoch [{epoch + 1}/{N_EPOCHS}], Evaluation Loss: {val_loss:.4f}, Evaluation BLEU: {val_bleu:.4f}')
    #val_loss, val_bleu, val_sari = evaluate_model(model, eval_data_loader, criterion, gpt2_tokenizer, ref_sentences[0])
    #print(f'Epoch [{epoch + 1}/{N_EPOCHS}], Evaluation Loss: {val_loss:.4f}, Evaluation BLEU: {val_bleu:.4f}, Evaluation SARI: {val_sari:.4f}')


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.3.crossattention.c_attn.weight', 'h.5.ln_cross_attn.weight', 'h.6.ln_cross_attn.weight', 'h.9.crossattention.q_attn.bias', 'h.9.ln_cross_attn.weight', 'h.10.crossattention.c_proj.bias', 'h.6.crossattention.q_attn.weight', 'h.6.crossattention.c_attn.bias', 'h.5.crossattention.c_attn.weight', 'h.3.ln_cross_attn.bias', 'h.7.crossattention.c_attn.bias', 'h.11.crossattention.c_attn.bias', 'h.1.crossattention.c_attn.weight', 'h.2.crossattention.c_attn.weight', 'h.4.crossattention.c_attn.bias', 'h.3.crossattention.c_attn.bias', 'h.11.crossattention.q_attn.bias', 'h.4.ln_cross_attn.weight', 'h.0.ln_cross_attn.bias', 'h.5.crossattention.q_attn.weight', 'h.7.crossattention.q_attn.bias', 'h.5.crossattention.c_proj.bias', 'h.1.ln_cross_attn.weight', 'h.4.crossattention.q_attn.bias', 'h.1.crossattention.c_proj.weight', 'h.10.crossattention.c_proj.weight', 'h.2.crossattention.q_attn.

Epoch [1/2000], Evaluation Loss: 1.3135, Evaluation BLEU: 0.0000
Epoch [2/2000], Evaluation Loss: 4.9308, Evaluation BLEU: 0.0000
Epoch [3/2000], Evaluation Loss: 8.6167, Evaluation BLEU: 0.0000
Epoch [4/2000], Evaluation Loss: 8.1539, Evaluation BLEU: 0.0000
Epoch [5/2000], Evaluation Loss: 9.7744, Evaluation BLEU: 0.0000
Epoch [6/2000], Evaluation Loss: 5.3611, Evaluation BLEU: 0.0000
Epoch [7/2000], Evaluation Loss: 5.2928, Evaluation BLEU: 0.0000
Epoch [8/2000], Evaluation Loss: 13.3926, Evaluation BLEU: 0.0000
Epoch [9/2000], Evaluation Loss: 6.7120, Evaluation BLEU: 0.0000
Epoch [10/2000], Evaluation Loss: 10.0990, Evaluation BLEU: 0.0000
Epoch [11/2000], Evaluation Loss: 9.5260, Evaluation BLEU: 0.0000
Epoch [12/2000], Evaluation Loss: 8.7591, Evaluation BLEU: 0.0000
Epoch [13/2000], Evaluation Loss: 8.6172, Evaluation BLEU: 0.0000
Epoch [14/2000], Evaluation Loss: 9.0559, Evaluation BLEU: 0.0000
Epoch [15/2000], Evaluation Loss: 9.2633, Evaluation BLEU: 0.0000
Epoch [16/2000], 

Epoch [124/2000], Evaluation Loss: 22.6129, Evaluation BLEU: 0.0000
Epoch [125/2000], Evaluation Loss: 22.6177, Evaluation BLEU: 0.0000
Epoch [126/2000], Evaluation Loss: 22.6225, Evaluation BLEU: 0.0000
Epoch [127/2000], Evaluation Loss: 22.6271, Evaluation BLEU: 0.0000
Epoch [128/2000], Evaluation Loss: 22.6316, Evaluation BLEU: 0.0000
Epoch [129/2000], Evaluation Loss: 22.6359, Evaluation BLEU: 0.0000
Epoch [130/2000], Evaluation Loss: 22.6401, Evaluation BLEU: 0.0000
Epoch [131/2000], Evaluation Loss: 22.6442, Evaluation BLEU: 0.0000
Epoch [132/2000], Evaluation Loss: 22.6482, Evaluation BLEU: 0.0000
Epoch [133/2000], Evaluation Loss: 22.6521, Evaluation BLEU: 0.0000
Epoch [134/2000], Evaluation Loss: 22.6558, Evaluation BLEU: 0.0000
Epoch [135/2000], Evaluation Loss: 22.6595, Evaluation BLEU: 0.0000
Epoch [136/2000], Evaluation Loss: 22.6632, Evaluation BLEU: 0.0000
Epoch [137/2000], Evaluation Loss: 22.6668, Evaluation BLEU: 0.0000
Epoch [138/2000], Evaluation Loss: 22.6703, Eval

Epoch [245/2000], Evaluation Loss: 22.9237, Evaluation BLEU: 0.0000
Epoch [246/2000], Evaluation Loss: 22.9253, Evaluation BLEU: 0.0000
Epoch [247/2000], Evaluation Loss: 22.9268, Evaluation BLEU: 0.0000
Epoch [248/2000], Evaluation Loss: 22.9283, Evaluation BLEU: 0.0000
Epoch [249/2000], Evaluation Loss: 22.9299, Evaluation BLEU: 0.0000
Epoch [250/2000], Evaluation Loss: 22.9314, Evaluation BLEU: 0.0000
Epoch [251/2000], Evaluation Loss: 22.9329, Evaluation BLEU: 0.0000
Epoch [252/2000], Evaluation Loss: 22.9344, Evaluation BLEU: 0.0000
Epoch [253/2000], Evaluation Loss: 22.9359, Evaluation BLEU: 0.0000
Epoch [254/2000], Evaluation Loss: 22.9373, Evaluation BLEU: 0.0000
Epoch [255/2000], Evaluation Loss: 22.9388, Evaluation BLEU: 0.0000
Epoch [256/2000], Evaluation Loss: 22.9403, Evaluation BLEU: 0.0000
Epoch [257/2000], Evaluation Loss: 22.9417, Evaluation BLEU: 0.0000
Epoch [258/2000], Evaluation Loss: 22.9431, Evaluation BLEU: 0.0000
Epoch [259/2000], Evaluation Loss: 22.9446, Eval

Epoch [366/2000], Evaluation Loss: 23.0615, Evaluation BLEU: 0.0000
Epoch [367/2000], Evaluation Loss: 23.0624, Evaluation BLEU: 0.0000
Epoch [368/2000], Evaluation Loss: 23.0632, Evaluation BLEU: 0.0000
Epoch [369/2000], Evaluation Loss: 23.0640, Evaluation BLEU: 0.0000
Epoch [370/2000], Evaluation Loss: 23.0648, Evaluation BLEU: 0.0000
Epoch [371/2000], Evaluation Loss: 23.0657, Evaluation BLEU: 0.0000
Epoch [372/2000], Evaluation Loss: 23.0665, Evaluation BLEU: 0.0000
Epoch [373/2000], Evaluation Loss: 23.0673, Evaluation BLEU: 0.0000
Epoch [374/2000], Evaluation Loss: 23.0681, Evaluation BLEU: 0.0000
Epoch [375/2000], Evaluation Loss: 23.0689, Evaluation BLEU: 0.0000
Epoch [376/2000], Evaluation Loss: 23.0697, Evaluation BLEU: 0.0000
Epoch [377/2000], Evaluation Loss: 23.0705, Evaluation BLEU: 0.0000
Epoch [378/2000], Evaluation Loss: 23.0713, Evaluation BLEU: 0.0000
Epoch [379/2000], Evaluation Loss: 23.0721, Evaluation BLEU: 0.0000
Epoch [380/2000], Evaluation Loss: 23.0729, Eval

...

Epoch [1448/2000], Evaluation Loss: 23.4377, Evaluation BLEU: 0.0000
Epoch [1449/2000], Evaluation Loss: 23.4379, Evaluation BLEU: 0.0000
Epoch [1450/2000], Evaluation Loss: 23.4380, Evaluation BLEU: 0.0000
Epoch [1451/2000], Evaluation Loss: 23.4382, Evaluation BLEU: 0.0000
Epoch [1452/2000], Evaluation Loss: 23.4384, Evaluation BLEU: 0.0000
Epoch [1453/2000], Evaluation Loss: 23.4385, Evaluation BLEU: 0.0000
Epoch [1454/2000], Evaluation Loss: 23.4387, Evaluation BLEU: 0.0000
Epoch [1455/2000], Evaluation Loss: 23.4389, Evaluation BLEU: 0.0000
Epoch [1456/2000], Evaluation Loss: 23.4391, Evaluation BLEU: 0.0000
Epoch [1457/2000], Evaluation Loss: 23.4393, Evaluation BLEU: 0.0000
Epoch [1458/2000], Evaluation Loss: 23.4395, Evaluation BLEU: 0.0000
Epoch [1459/2000], Evaluation Loss: 23.4397, Evaluation BLEU: 0.0000
Epoch [1460/2000], Evaluation Loss: 23.4398, Evaluation BLEU: 0.0000
Epoch [1461/2000], Evaluation Loss: 23.4400, Evaluation BLEU: 0.0000
Epoch [1462/2000], Evaluation Loss

Epoch [1567/2000], Evaluation Loss: 23.4455, Evaluation BLEU: 0.0000
Epoch [1568/2000], Evaluation Loss: 23.4458, Evaluation BLEU: 0.0000
Epoch [1569/2000], Evaluation Loss: 23.4461, Evaluation BLEU: 0.0000
Epoch [1570/2000], Evaluation Loss: 23.4463, Evaluation BLEU: 0.0000
Epoch [1571/2000], Evaluation Loss: 23.4465, Evaluation BLEU: 0.0000
Epoch [1572/2000], Evaluation Loss: 23.4467, Evaluation BLEU: 0.0000
Epoch [1573/2000], Evaluation Loss: 23.4468, Evaluation BLEU: 0.0000
Epoch [1574/2000], Evaluation Loss: 23.4470, Evaluation BLEU: 0.0000
Epoch [1575/2000], Evaluation Loss: 23.4471, Evaluation BLEU: 0.0000
Epoch [1576/2000], Evaluation Loss: 23.4473, Evaluation BLEU: 0.0000
Epoch [1577/2000], Evaluation Loss: 23.4476, Evaluation BLEU: 0.0000
Epoch [1578/2000], Evaluation Loss: 23.4478, Evaluation BLEU: 0.0000
Epoch [1579/2000], Evaluation Loss: 23.4481, Evaluation BLEU: 0.0000
Epoch [1580/2000], Evaluation Loss: 23.4483, Evaluation BLEU: 0.0000
Epoch [1581/2000], Evaluation Loss

Epoch [1686/2000], Evaluation Loss: 23.4604, Evaluation BLEU: 0.0000
Epoch [1687/2000], Evaluation Loss: 23.4604, Evaluation BLEU: 0.0000
Epoch [1688/2000], Evaluation Loss: 23.4604, Evaluation BLEU: 0.0000
Epoch [1689/2000], Evaluation Loss: 23.4605, Evaluation BLEU: 0.0000
Epoch [1690/2000], Evaluation Loss: 23.4606, Evaluation BLEU: 0.0000
Epoch [1691/2000], Evaluation Loss: 23.4607, Evaluation BLEU: 0.0000
Epoch [1692/2000], Evaluation Loss: 23.4608, Evaluation BLEU: 0.0000
Epoch [1693/2000], Evaluation Loss: 23.4609, Evaluation BLEU: 0.0000
Epoch [1694/2000], Evaluation Loss: 23.4611, Evaluation BLEU: 0.0000
Epoch [1695/2000], Evaluation Loss: 23.4612, Evaluation BLEU: 0.0000
Epoch [1696/2000], Evaluation Loss: 23.4614, Evaluation BLEU: 0.0000
Epoch [1697/2000], Evaluation Loss: 23.4617, Evaluation BLEU: 0.0000
Epoch [1698/2000], Evaluation Loss: 23.4619, Evaluation BLEU: 0.0000
Epoch [1699/2000], Evaluation Loss: 23.4621, Evaluation BLEU: 0.0000
Epoch [1700/2000], Evaluation Loss

Epoch [1805/2000], Evaluation Loss: 23.4862, Evaluation BLEU: 0.0000
Epoch [1806/2000], Evaluation Loss: 23.4863, Evaluation BLEU: 0.0000
Epoch [1807/2000], Evaluation Loss: 23.4864, Evaluation BLEU: 0.0000
Epoch [1808/2000], Evaluation Loss: 23.4864, Evaluation BLEU: 0.0000
Epoch [1809/2000], Evaluation Loss: 23.4864, Evaluation BLEU: 0.0000
Epoch [1810/2000], Evaluation Loss: 23.4863, Evaluation BLEU: 0.0000
Epoch [1811/2000], Evaluation Loss: 23.4862, Evaluation BLEU: 0.0000
Epoch [1812/2000], Evaluation Loss: 23.4861, Evaluation BLEU: 0.0000
Epoch [1813/2000], Evaluation Loss: 23.4860, Evaluation BLEU: 0.0000
Epoch [1814/2000], Evaluation Loss: 23.4859, Evaluation BLEU: 0.0000
Epoch [1815/2000], Evaluation Loss: 23.4857, Evaluation BLEU: 0.0000
Epoch [1816/2000], Evaluation Loss: 23.4855, Evaluation BLEU: 0.0000
Epoch [1817/2000], Evaluation Loss: 23.4854, Evaluation BLEU: 0.0000
Epoch [1818/2000], Evaluation Loss: 23.4852, Evaluation BLEU: 0.0000
Epoch [1819/2000], Evaluation Loss

Epoch [1924/2000], Evaluation Loss: 23.4975, Evaluation BLEU: 0.0000
Epoch [1925/2000], Evaluation Loss: 23.4975, Evaluation BLEU: 0.0000
Epoch [1926/2000], Evaluation Loss: 23.4975, Evaluation BLEU: 0.0000
Epoch [1927/2000], Evaluation Loss: 23.4976, Evaluation BLEU: 0.0000
Epoch [1928/2000], Evaluation Loss: 23.4976, Evaluation BLEU: 0.0000
Epoch [1929/2000], Evaluation Loss: 23.4976, Evaluation BLEU: 0.0000
Epoch [1930/2000], Evaluation Loss: 23.4976, Evaluation BLEU: 0.0000
Epoch [1931/2000], Evaluation Loss: 23.4976, Evaluation BLEU: 0.0000
Epoch [1932/2000], Evaluation Loss: 23.4976, Evaluation BLEU: 0.0000
Epoch [1933/2000], Evaluation Loss: 23.4976, Evaluation BLEU: 0.0000
Epoch [1934/2000], Evaluation Loss: 23.4976, Evaluation BLEU: 0.0000
Epoch [1935/2000], Evaluation Loss: 23.4976, Evaluation BLEU: 0.0000
Epoch [1936/2000], Evaluation Loss: 23.4976, Evaluation BLEU: 0.0000
Epoch [1937/2000], Evaluation Loss: 23.4977, Evaluation BLEU: 0.0000
Epoch [1938/2000], Evaluation Loss

In [170]:
data = WikiDataset.open_file("../dataset/src_valid.txt")

data[10]


'A hunting dog refers to any dog who assists humans in hunting .'